In [ ]:
cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import segmentation_models_pytorch as smp
import os
import random

# --- 1. CẤU HÌNH HÀM XỬ LÝ ẢNH ---
# Hàm này giúp biến đổi ảnh thường thành dạng mà Model ResNet34 hiểu được
preprocessing_fn = smp.encoders.get_preprocessing_fn('resnet34', 'imagenet')

def test_single_image(image_path, model, device):
    # --- BƯỚC 1: ĐỌC VÀ XỬ LÝ ẢNH ---
    # Đọc ảnh từ đường dẫn
    image = cv2.imread(image_path)
    if image is None:
        print(f"Lỗi: Không tìm thấy ảnh tại '{image_path}'")
        return

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  
    # Resize về 512x512 (BẮT BUỘC - Vì lúc train dùng size này)
    # Nếu không resize, model sẽ báo lỗi kích thước
    image_resized = cv2.resize(image, (512, 512))
  
    # Preprocess (Chuẩn hóa màu sắc theo chuẩn ResNet)
    input_image = preprocessing_fn(image_resized)
  
    # Chuyển sang Tensor: (H, W, C) -> (C, H, W)
    input_tensor = torch.from_numpy(input_image.transpose(2, 0, 1)).float()
  
    # Thêm chiều Batch: (1, C, H, W) và đưa lên GPU/CPU
    input_tensor = input_tensor.unsqueeze(0).to(device)
  
    # --- BƯỚC 2: DỰ ĐOÁN (INFERENCE) ---
    model.to(device)
    model.eval() # Chế độ test (tắt dropout/batchnorm động)
  
    with torch.no_grad():
        prediction = model(input_tensor)
  
    # Kết quả trả về là xác suất, dùng sigmoid nếu model chưa có, 
    # nhưng đã khai báo activation='sigmoid' rồi nên nó đã là 0-1.
    # Tuy nhiên để chắc ăn ta cứ convert sang numpy để xử lý.
    pred_mask = prediction.squeeze().cpu().numpy()
  
    # Chuyển về nhị phân: Lớn hơn 0.5 là Rừng (1), còn lại là Nền (0)
    # (có thể chỉnh số 0.5 này lên xuống để lọc nhiễu)
    binary_mask = (pred_mask > 0.5).astype(np.uint8)

    # --- BƯỚC 3: HIỂN THỊ KẾT QUẢ ---
    plt.figure(figsize=(18, 6))

    # Hình 1: Ảnh gốc
    plt.subplot(1, 3, 1)
    plt.imshow(image_resized)
    plt.title("Ảnh Thực Tế (512x512)")
    plt.axis('off')

    # Hình 2: Mask AI dự đoán (Đen trắng)
    plt.subplot(1, 3, 2)
    plt.imshow(binary_mask, cmap='gray')
    plt.title("AI Dự Đoán (Trắng = Rừng)")
    plt.axis('off')

    # Hình 3: Mask chồng lên ảnh (Overlay) - Dễ nhìn nhất
    plt.subplot(1, 3, 3)
    plt.imshow(image_resized)
    # Phủ màu xanh lá lên vùng rừng
    plt.imshow(binary_mask, cmap='jet', alpha=0.5) 
    plt.title("Kết quả chồng lên ảnh")
    plt.axis('off')

    plt.show()

# --- HÀM TEST VÀ TÁCH NỀN TRẮNG ---
def visualize_forest_on_white(image_path, model, device):
    # 1. Đọc và xử lý ảnh đầu vào
    image = cv2.imread(image_path)
    if image is None:
        print("Không tìm thấy ảnh!")
        return
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  
    # Resize về 512x512 (Kích thước model đã học)
    image_resized = cv2.resize(image, (512, 512))
  
    # Chuẩn bị đưa vào model
    preprocessing_fn = smp.encoders.get_preprocessing_fn('resnet34', 'imagenet')
    input_image = preprocessing_fn(image_resized)
    input_tensor = torch.from_numpy(input_image.transpose(2, 0, 1)).float().unsqueeze(0).to(device)
  
    # 2. Dự đoán (Lấy Mask)
    model.eval()
    with torch.no_grad():
        prediction = model(input_tensor)
  
    # Chuyển về mask nhị phân (0 và 1)
    pred_mask = prediction.squeeze().cpu().numpy()
    binary_mask = (pred_mask > 0.5).astype(np.uint8) # 1 là Rừng, 0 là Nền
  
    # 3. --- KỸ THUẬT TẠO ẢNH NỀN TRẮNG ---
  
    # Tạo một bức ảnh trắng tinh cùng kích thước (512x512)
    # 255 là màu trắng trong ảnh 8-bit
    white_background = np.ones_like(image_resized) * 255
  
    # Tạo mask 3 kênh để khớp với ảnh màu (vì mask hiện tại chỉ có 1 kênh)
    mask_3ch = np.stack([binary_mask]*3, axis=-1)
  
    # Công thức: (Ảnh Gốc * Mask) + (Ảnh Trắng * Đảo Ngược Mask)
    # - Phần Rừng (Mask=1): Giữ nguyên ảnh gốc
    # - Phần Nền (Mask=0): Lấy màu trắng
    final_result = np.where(mask_3ch == 1, image_resized, white_background)
  
    # 4. Hiển thị kết quả
    plt.figure(figsize=(12, 6))
  
    plt.subplot(1, 2, 1)
    plt.imshow(image_resized)
    plt.title("Ảnh Gốc (Có đường, nhà...)")
    plt.axis('off')
  
    plt.subplot(1, 2, 2)
    plt.imshow(final_result)
    plt.title("Kết quả: Chỉ giữ Rừng (Nền trắng)")
    plt.axis('off')
  
    plt.show()
  
    # (Tùy chọn) Lưu ảnh ra file nếu muốn
    # cv2.imwrite('ket_qua_tach_rung.jpg', cv2.cvtColor(final_result, cv2.COLOR_RGB2BGR))


# --- CHẠY TEST ---
# path để test
folder_path = '/content/drive/MyDrive/deepglobe_land/train'

# 1. Lấy danh sách tất cả các file ảnh vệ tinh (có đuôi _sat.jpg)
# Cách này giúp tránh bốc nhầm file mask hoặc file rác
all_images = [f for f in os.listdir(folder_path) if f.endswith('_sat.jpg')]

# Kiểm tra xem có tìm thấy ảnh không
model.to(DEVICE)
for i in range(0,10):
  if len(all_images) > 0:
      # 2. Chọn bừa 1 cái tên trong danh sách
      random_image_name = random.choice(all_images)
  
      # 3. Tạo đường dẫn đầy đủ
      random_image_path = os.path.join(folder_path, random_image_name)
  
      print(f"Đang test ngẫu nhiên ảnh: {random_image_name}")
  
      # 4. Gọi hàm test cũ của bạn
      visualize_forest_on_white(random_image_path, model, DEVICE)
      test_single_image(random_image_path, model, DEVICE)
  else:
      print("Không tìm thấy ảnh nào có đuôi '_sat.jpg' trong thư mục này!")

